In [1]:
!pip install transformers
!pip install datasets
!pip install --upgrade gensim
import nltk
nltk.download('wordnet')

     |████████████████████████████████| 4.0 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 51.5 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 6.6 MB 49.4 MB/s 
     |████████████████████████████████| 596 kB 63.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 3.5 MB/s 
     |████████████████████████████████| 136 kB 53.8 MB/s 
     |████████████████████████████████| 212 kB 43.6 MB/s 
     |████████████████████████████████| 1.1 MB 62.8 MB/s 
     |████████████████████████████████| 127 kB 39.0 MB/s 
     |████████████████████████████████| 144 kB 27.4 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 42.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

True

In [5]:
from google.colab import drive
import pandas as pd 
import gensim
from gensim.models import Word2Vec


drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###Data

###load data


In [ ]:
dataframe=pd.read_csv('gdrive/My Drive/thesis/version4/Concepts_Clean_v1.csv')
dataframe

In [ ]:
from sklearn.model_selection import train_test_split
del dataframe['package']

train, test = train_test_split(dataframe, test_size=0.3)
train.to_csv('gdrive/My Drive/thesis/version4/train_.csv',index=False)
test.to_csv('gdrive/My Drive/thesis/version4/test_.csv',index=False)

###distilgpt2

In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv",data_files={"train":"gdrive/My Drive/thesis/version4/train_.csv", "test":"gdrive/My Drive/thesis/version4/test_.csv"})
dataset

Using custom data configuration default-ca88c3f2c203506a


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ca88c3f2c203506a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sequences'],
        num_rows: 103563
    })
    test: Dataset({
        features: ['sequences'],
        num_rows: 44385
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("distilgpt2")


Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

#generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [ ]:

def tokenize_function(examples):
    return tokenizer(examples["sequences"],padding=True, truncation=True, max_length=128)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True,load_from_cache_file=False)

  0%|          | 0/104 [00:00<?, ?ba/s]

  0%|          | 0/45 [00:00<?, ?ba/s]

In [ ]:

tokenized_dataset['train'] = tokenized_dataset['train'].add_column("labels", tokenized_dataset['train']['input_ids'])

tokenized_dataset['test'] = tokenized_dataset['test'].add_column("labels", tokenized_dataset['test']['input_ids'])
tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['sequences', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 103563
    })
    test: Dataset({
        features: ['sequences', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 44385
    })
})

In [ ]:
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from transformers import TrainingArguments, Trainer,default_data_collator


training_args = TrainingArguments(
    output_dir="gdrive/My Drive/thesis/version4/results2",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
  
    save_strategy="epoch",
    do_train=True,
    do_eval=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    
)

trainer.train()
trainer.save_model('gdrive/My Drive/thesis/version4/fineTunedModelConceptsPredictionLowerCase')

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: sequence. If sequence are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 48996
  Batch size = 32


###Retrain for a second time (third epoch)

In [ ]:
from transformers import TrainingArguments, Trainer,default_data_collator


training_args = TrainingArguments(
    output_dir="gdrive/My Drive/thesis/data/results_Sequents/thirdEpoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
  
    save_strategy="epoch",
    do_train=True,
    do_eval=True
)

trainer = Trainer(
    model=TrainedModel, ### I change this config !
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    
)

trainer.train()
#trainer.save_model('gdrive/My Drive/thesis/fineTunedModelWithSubSequents')

###Test the model

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

TrainedModel = AutoModelForCausalLM.from_pretrained("gdrive/My Drive/thesis/version4/fineTunedModelConceptsPredictionLowerCase")


Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

##### ngram_size ( set to 5 and not 2 because we considere parentheses) 
it is ok to have the class name repeated in other triplets. 

In [ ]:
input_ids = tokenizer('(employee, company)', return_tensors='pt')
beam_output = TrainedModel.generate(**input_ids,
   max_length=200, #or 100
   
   no_repeat_ngram_size=3, 
   early_stopping=True
)
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


(employee, company)', '(employeemanager,employee)', 'employeemanagement,employeereport)',


In [ ]:
input_ids = tokenizer('(driver, car) ', return_tensors='pt')
beam_output = TrainedModel.generate(**input_ids,
   max_length=100, 
  
   no_repeat_ngram_size=5, 
   early_stopping=True
)
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


(driver, car) vernier,driver)', '(passenger,driver)', '[session,driver)', ('vehicle,carpoolingdriver)', '(vehicledescriptor,driver)'


###Validation

In [3]:
import pickle
loaded_model = pickle.load(open('gdrive/My Drive/thesis/predictionAssociationo_Inheritance/RandomForest_model2.sav','rb'))

In [6]:
import re
def intercept(Input):

  input_ids = tokenizer(Input, return_tensors='pt')
  beam_output = TrainedModel.generate(**input_ids,
   max_length=200, #or 100
   
   no_repeat_ngram_size=5, 
   early_stopping=True
  )


  result=tokenizer.decode(beam_output[0], skip_special_tokens=True)
  print(result)
  new=result.split(Input)[1]
  print(new)
  results=[]
 
  while True: 
    try:
      i=re.search(r'\((.*?)\)',new).group(1)
      results.append(i)
      
      new=new.split(i)[1]
    except: 
      break
  ##check no repeated association: 
  print(results)
  assert  len(results) == len(set(results)), 'not_repeated classes'
  #for i in results: 

    





SyntaxError: ignored

In [ ]:
intercept('(car, driver)')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


(car, driver)', '(car,vehicle)', '(carpool,vehicle)
', '(car,vehicle)', '(carpool,vehicle)
['car,vehicle', 'carpool,vehicle']


In [7]:
from gensim.models import KeyedVectors

Vectorizer=KeyedVectors.load_word2vec_format("gdrive/My Drive/thesis/embeded/pretrained/vectorizer")

In [8]:
import numpy as np
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      print(tokens)
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [9]:
import re
import nltk
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and     characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [13]:
test='(plant, driver)'

In [14]:
test2=utils_preprocess_text(test)

In [15]:
res=Word2VecVectorizer(Vectorizer).transform([test2])
print(len(res[0]))
loaded_model.predict(res)

Loading in word vectors...
Finished loading in word vectors
['plant', 'driver']
Numer of samples with no words found: 0 / 1
300


array([0])